In [ ]:
!pip install datasets
!pip install tokenizers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.5 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 212 kB 66.0 MB/s 
     |████████████████████████████████| 141 kB 62.7 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 127 kB 72.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 w

loading the data set from wikiann data files

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "hi")

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/hi/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans'],
    num_rows: 5000
})

checkin the features of wikiann ner data set

In [ ]:
label_names = dataset["train"].features["ner_tags"].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

Tokenizing the input data in appropriate format for training

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

#Get the values for input_ids, token_type_ids, attention_mask

def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  print(len(tokenized_samples["input_ids"]))
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []
    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])
        
    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

1000


  0%|          | 0/1 [00:00<?, ?ba/s]

1000


  0%|          | 0/5 [00:00<?, ?ba/s]

1000
1000
1000
1000
1000


In [ ]:
tokenized_dataset["validation"][0]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 20321, 15070, 44302, 872, 90696, 76380, 99362, 65781, 102],
 'labels': [-100, 3, 3, 3, 4, 4, 4, 4, 4, -100],
 'langs': ['hi', 'hi', 'hi', 'hi'],
 'ner_tags': [3, 4, 4, 4],
 'spans': ['ORG: हावड़ा जंक्शन रेलवे स्टेशन'],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['हावड़ा', 'जंक्शन', 'रेलवे', 'स्टेशन']}

padding for the input data so the model can expect same length for all input data points

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

setting up WANDDB for displaying results and tracking evaluation metrices

In [ ]:
!pip install wandb
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 15.3 MB/s 
     |████████████████████████████████| 156 kB 75.2 MB/s 
     |████████████████████████████████| 181 kB 71.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=657dd3259b453890f8eaa59fcb2259d02b356bc37eb4966b54b9efe9811d78ae
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=8379123b2e0184d1d47e98f743e155cd6d95160f6327490591acc7ac293e61be
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e315

setting up wanddb to see evaluation metrices

In [ ]:
import os
import wandb
os.environ["WANDB_API_KEY"]="xxxxxxxxxxxxxxxx"
wandb.init(project="hindi_tagger_bert", entity="lakshay_7")

writing metrices functions to understand output for evaluation

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_names))
training_args = TrainingArguments(
    output_dir="./ner_hindi_bert",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="wandb",
    run_name = "ep_10_tokenized_11",
    save_strategy='no'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embedding

Step,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Loc F1,Org F1,Per F1
1000,0.299300,0.323026,0.877889,0.878581,0.878235,0.924402,0.853488,0.826988,0.935767
2000,0.064100,0.371294,0.894185,0.897240,0.895710,0.936651,0.876586,0.848865,0.945442


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: langs, ner_tags, spans, tokens. If langs, ner_tags, spans, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: langs, ner_tags, spans, tokens. If langs, ner_tags, spans, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




eval/LOC_f1,▁█
eval/ORG_f1,▁█
eval/PER_f1,▁█
eval/loss,▁█
eval/overall_accuracy,▁█
eval/overall_f1,▁█
eval/overall_precision,▁█
eval/overall_recall,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█


checking metrices on testing data set.

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)
#Here we will see that the labels list will have lots of -100 in them however the corresponding label of the 
#tokenized_dataset doesnt have it, the reason is during DataCollator padding step, all padding tokens are added
#and assigned labels of -100 to get "ignored" in future computation of evaluation

# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: langs, ner_tags, spans, tokens. If langs, ner_tags, spans, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 16


{'LOC': {'f1': 0.8669705760464153,
  'number': 1195,
  'precision': 0.8587848932676518,
  'recall': 0.8753138075313808},
 'ORG': {'f1': 0.8903290129611166,
  'number': 986,
  'precision': 0.8754901960784314,
  'recall': 0.9056795131845842},
 'PER': {'f1': 0.9308848080133555,
  'number': 1481,
  'precision': 0.9207397622192867,
  'recall': 0.9412559081701553},
 'overall_accuracy': 0.9397143257246884,
 'overall_f1': 0.8991097922848664,
 'overall_precision': 0.8883262260127932,
 'overall_recall': 0.9101583833970508}

saving the ner marathi model

In [ ]:
model.save_pretrained("ner_hindi")

Configuration saved in ner_hindi/config.json
Model weights saved in ner_hindi/pytorch_model.bin


In [ ]:
saved_model = AutoModelForTokenClassification.from_pretrained("ner_hindi")
saved_model

loading configuration file ner_hindi/config.json
Model config BertConfig {
  "_name_or_path": "ner_hindi",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

checking the model on new data

In [ ]:
# import torch
# random_sentence_from_internet = ["राज्यसभा","निवडणुकांसाठी","मुंबईत","भाजपचे" ,"चिंचवडचे", "आमदार", "लक्ष्मण", "जगताप"]  
# input = tokenizer(random_sentence_from_internet, is_split_into_words=True, return_tensors='pt')
# print(input)
# output = saved_model(**input)
# predictions = torch.nn.functional.softmax(output.logits, dim=-1)
# predictions = predictions.detach().numpy()
# predictions = np.argmax(predictions, axis=2)
# print(predictions)

# pred_names = [label_names[p] for p in predictions[0]]
# for index, id in enumerate(input["input_ids"][0]):
#   print("\nID: ", id, "Decoded ID: ", tokenizer.decode(id), "\tPred: ", pred_names[index])

{'input_ids': tensor([[   101,  16916,  13432, 104741,    884,  66306,  20691,  14512,  97571,
          11208,  49096,  86843,  65800,  11845,    888,  44735,  18187,  19030,
            870, 103435,  16940,  15070,  20691,  19030,  70575,  55904,    893,
          32662,  89733,    872,  49824,  89150,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}
[[2 3 3 3 0 0 0 0 0 0 0 0 5 5 0 0 0 0 0 4 4 4 4 4 0 0 1 1 1 2 2 2 2]]

ID:  tensor(101) Decoded ID:  [CLS] 	Pred:  I-PER

ID:  tensor(16916) Decoded ID:  राज्य 	Pred:  B-ORG

ID:  tensor(13432) Decoded ID:  ##स 	Pred:  B-ORG

ID:  tensor(104741) Decoded ID:  ##भा 	Pred:  B-ORG

ID:  tensor(884) Decoded ID:  न 	Pred:  O

ID:  tensor(66306) Decoded ID:  ##िव 	Pred:  O

ID:  tensor(20691) Decoded ID:  ##ड 	

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
trainer.push_to_hub("NER_HINDI_BERT")

Cloning https://huggingface.co/lakshaywadhwa1993/ner_hindi_bert into local empty directory.
Saving model checkpoint to ./ner_hindi_bert
Configuration saved in ./ner_hindi_bert/config.json
Model weights saved in ./ner_hindi_bert/pytorch_model.bin
tokenizer config file saved in ./ner_hindi_bert/tokenizer_config.json
Special tokens file saved in ./ner_hindi_bert/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.34k/676M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.23k/3.23k [00:00<?, ?B/s]

To https://huggingface.co/lakshaywadhwa1993/ner_hindi_bert
   bc2e4e9..af18aca  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'dataset': {'name': 'wikiann', 'type': 'wikiann', 'config': 'hi', 'split': 'train', 'args': 'hi'}}
To https://huggingface.co/lakshaywadhwa1993/ner_hindi_bert
   af18aca..1a2b6d5  main -> main



'https://huggingface.co/lakshaywadhwa1993/ner_hindi_bert/commit/af18aca5acd4af834c37c04ed34c2b7e2bbd0df6'

In [ ]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline

# tokenizer = AutoTokenizer.from_pretrained("lakshaywadhwa1993/ner_marathi_bert")

# model = AutoModelForTokenClassification.from_pretrained("lakshaywadhwa1993/ner_marathi_bert")

# nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# example = ["राज्यसभा","निवडणुकांसाठी","मुंबईत","भाजपचे" ,"चिंचवडचे", "आमदार", "लक्ष्मण", "जगताप"]

# results = nlp(example)

# results

[[{'end': 5,
   'entity': 'LABEL_4',
   'index': 1,
   'score': 0.65346557,
   'start': 0,
   'word': 'राज्य'},
  {'end': 6,
   'entity': 'LABEL_4',
   'index': 2,
   'score': 0.9602548,
   'start': 5,
   'word': '##स'},
  {'end': 8,
   'entity': 'LABEL_4',
   'index': 3,
   'score': 0.97620004,
   'start': 6,
   'word': '##भा'}],
 [{'end': 1,
   'entity': 'LABEL_4',
   'index': 1,
   'score': 0.98540753,
   'start': 0,
   'word': 'न'},
  {'end': 3,
   'entity': 'LABEL_4',
   'index': 2,
   'score': 0.98875844,
   'start': 1,
   'word': '##िव'},
  {'end': 4,
   'entity': 'LABEL_4',
   'index': 3,
   'score': 0.9879582,
   'start': 3,
   'word': '##ड'},
  {'end': 5,
   'entity': 'LABEL_4',
   'index': 4,
   'score': 0.98931855,
   'start': 4,
   'word': '##ण'},
  {'end': 7,
   'entity': 'LABEL_4',
   'index': 5,
   'score': 0.98987234,
   'start': 5,
   'word': '##ुक'},
  {'end': 8,
   'entity': 'LABEL_4',
   'index': 6,
   'score': 0.9930636,
   'start': 7,
   'word': '##ा'},
  {'end':